DATASET USED: 

[Khan, Muhammad Yaseen, and Nizami, Muhammad Suffian. "Urdu Sentiment Corpus (v1.0): Linguistic Exploration and Visualization of Labeled Datasetfor Urdu Sentiment Analysis." In 2020 IEEE 2nd International Conference On Information Science & Communication Technology (ICISCT). IEEE, 2020.](https://ieeexplore.ieee.org/abstract/document/9080043/)

In [161]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, LSTM, GRU
from sklearn.model_selection import train_test_split

# Preprocessing

In [ ]:
df = pd.read_csv('C:/Users/Dell/Desktop/urdu-sentiment-corpus-v1.tsv', sep='\t')
df

,Tweet,Class
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P
...,...,...
995,اُس آدمی نے اِس سالار کو کافی معقول ٹپ دی ہے ۔,P
996,چچا غالب کی روح سے معذرت کے ساتھہم نے مانا کہ ...,P
997,واہ جناب واہ! اچھی رہی۔ جناب خود کو فرشتہ سمجو...,P
998,اسلام آباد :پی اے ٹی کا دھرنا ختم، صفائی کے کا...,P


In [ ]:
df = df.dropna()
df['Class'].unique()

array(['P', 'N', 'O'], dtype=object)

In [ ]:
print('O', (df['Class'] == 'O').sum())
print('N', (df['Class'] == 'N').sum())
print('P', (df['Class'] == 'P').sum())

O 20
N 499
P 480


In [ ]:
df = df[df['Class'] != 'O']
df['Class'].unique()

array(['P', 'N'], dtype=object)

In [ ]:
urdu_diacritics  = ['ِ', 'ٰ', 'ُ', 'ٍ', 'ً', 'َ']
def remove_diacritics(text):
    for letter in text:
    
        if letter in urdu_diacritics:
            text = text.replace(letter, '')
    return text

In [ ]:
def remove_punctuations(raw_text):
    preprocessed_text=re.sub(r'[؛۔٫٪+=@#!؟،۔)(}{]', '',raw_text)
    return preprocessed_text

In [ ]:
digits = ['۶', '۴', '۵', '۸', '۲', '۰', '۷', '۹', '۳', '۱','1','2','3','4','5','6','7','8','9','0']
def remove_digits(text):
    for letter in text:
        if letter in digits:
            text = text.replace(letter, '')
    return text

In [ ]:
def tokenize(text):
    tokens = text.split()
    return tokens

In [ ]:
stopwords=[]
with open('C:/Users/Dell/Downloads/stop_words.txt',encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
      stopwords.append(line.strip('\n'))

stopwords

['آ',
 'آؤ',
 'آو',
 'آئی',
 'آئیں',
 'آئے',
 'آج',
 'آجاؤ',
 'آجائیں',
 'آجائیے',
 'آجاو',
 'آجکل',
 'آپ',
 'آپکا',
 'آپکو',
 'آپکی',
 'آیا',
 'اب',
 'ابھی',
 'اس',
 'اسطرح',
 'اسکا',
 'اسکی',
 'اسکے',
 'اسی',
 'اسے',
 'اطراف',
 'الگ',
 'ان',
 'انکا',
 'انکی',
 'انکے',
 'انھوں',
 'انھیں',
 'انہیں',
 'اں',
 'اور',
 'اونچا',
 'اونچی',
 'اونچے',
 'اوپر',
 'اپنا',
 'اپنی',
 'اپنے',
 'اکثر',
 'اگر',
 'اگرچہ',
 'اہم',
 'بائیں',
 'بار',
 'بارے',
 'باہر',
 'باہم',
 'بذریعہ',
 'بس',
 'بظاہر',
 'بعد',
 'بغیر',
 'بلاشبہ',
 'بند',
 'بہت',
 'بھی',
 'بیشتر',
 'بیشک',
 'بےشک',
 'تاہم',
 'تب',
 'تجھ',
 'تجھے',
 'تر',
 'ترین',
 'تعداد',
 'تلک',
 'تم',
 'تمام',
 'تمکو',
 'تمھارا',
 'تمھاری',
 'تمھارے',
 'تمھیں',
 'تمہارا',
 'تمہاری',
 'تمہارے',
 'تمہیں',
 'تو',
 'تک',
 'تھا',
 'تھی',
 'تھیں',
 'تھے',
 'تیسرا',
 'تیسری',
 'تیسرے',
 'جائیں',
 'جاتا',
 'جاتی',
 'جاتے',
 'جانا',
 'جانے',
 'جائے',
 'جب',
 'جبہی',
 'جبھی',
 'جبکہ',
 'جو',
 'جہاں',
 'جیسا',
 'جیساکہ',
 'جیسی',
 'جیسے',
 'جیسےکہ',
 'خود',
 'دا

In [ ]:
def remove_stopwords(text):
    temp=[]
    words=text.split()
    for letter in words:
        if letter not in stopwords:
          if letter !='':
            temp.append(letter)
            
    return (' '.join(temp) )

In [ ]:
df['Tweet'] = df['Tweet'].apply(lambda x: remove_diacritics(x))
df['Tweet'] = df['Tweet'].apply(lambda x: remove_digits(x))
df['Tweet'] = df['Tweet'].apply(lambda x: remove_punctuations(x))
df['Tweet'] = df['Tweet'].apply(lambda x: remove_stopwords(x))
df['Tweet']

0      [ایٹم, بم, بنایا, ھے, او, بھائی, ایٹم, بمب, کو...
1             [چندے, انقلاب, عمران, خان, وزیر, اعظم, بن]
3      [سرچ, انجن, گوگل, نائب, صدر, فضا, ,, فٹ, بلندی...
4                                      [لہریں, یار, :أْ]
5      [گندی, زبان, گٹر, دماغ, جاهل, جیالے, هو, جیالا...
                             ...                        
995                       [آدمی, سالار, کافی, معقول, ٹپ]
996    [چچا, غالب, روح, معذرت, ساتھہم, مانا, ڈیزلمفت,...
997    [واہ, جناب, واہ, اچھی, جناب, فرشتہ, سمجوں, انسان]
998    [اسلام, آباد, :پی, اے, ٹی, دھرنا, ختم, صفائی, ...
999         [دنیا, راہ, وفا, ساتھتم, چلے, چلو, چلی, چلے]
Name: Tweet, Length: 979, dtype: object

In [ ]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stem_and_lemmatize(text):
    stemmed_words = [stemmer.stem(word) for word in text]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    return " ".join(lemmatized_words)

df['Tweet'] = df['Tweet'].apply(stem_and_lemmatize)

In [ ]:
df['Tweet']

0      ایٹم بم بنایا ھے او بھائی ایٹم بمب کوٹ لکھپت ا...
1                     چندے انقلاب عمران خان وزیر اعظم بن
3      سرچ انجن گوگل نائب صدر فضا , فٹ بلندی چھلانگ ع...
4                                          لہریں یار :أْ
5      گندی زبان گٹر دماغ جاهل جیالے هو جیالا هو جاهل...
                             ...                        
995                             آدمی سالار کافی معقول ٹپ
996      چچا غالب روح معذرت ساتھہم مانا ڈیزلمفت ہاتھ برا
997             واہ جناب واہ اچھی جناب فرشتہ سمجوں انسان
998    اسلام آباد :پی اے ٹی دھرنا ختم صفائی کام آغاز ...
999                  دنیا راہ وفا ساتھتم چلے چلو چلی چلے
Name: Tweet, Length: 979, dtype: object

In [ ]:
labelencoder = LabelEncoder()
df['Class'] = labelencoder.fit_transform(df['Class'])
print(df['Class'])

0      1
1      0
3      1
4      1
5      0
      ..
995    1
996    1
997    1
998    1
999    1
Name: Class, Length: 979, dtype: int32


In [ ]:
print('N', (df['Class'] == 0).sum())
print('P', (df['Class'] == 1).sum())

N 499
P 480


In [ ]:
df.dtypes

Tweet    object
Class     int32
dtype: object

In [134]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

X = df['Tweet']
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_seq_padded = pad_sequences(X_train_seq, 50)
X_test_seq_padded = pad_sequences(X_test_seq, 50)

# SimpleRNN

layers 2, drop-out 0.3

In [135]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 5s 69ms/step - loss: 0.7058 - acc: 0.5095 - val_loss: 0.6896 - val_acc: 0.4857
Epoch 2/10
23/23 [==============================] - 1s 26ms/step - loss: 0.5482 - acc: 0.7520 - val_loss: 0.6948 - val_acc: 0.4980
Epoch 3/10
23/23 [==============================] - 1s 26ms/step - loss: 0.2913 - acc: 0.9060 - val_loss: 0.7663 - val_acc: 0.5143
Epoch 4/10
23/23 [==============================] - 1s 26ms/step - loss: 0.1461 - acc: 0.9523 - val_loss: 0.8467 - val_acc: 0.4980
Epoch 5/10
23/23 [==============================] - 1s 27ms/step - loss: 0.0789 - acc: 0.9741 - val_loss: 0.9465 - val_acc: 0.5102
Epoch 6/10
23/23 [==============================] - 1s 27ms/step - loss: 0.0477 - acc: 0.9864 - val_loss: 1.0867 - val_acc: 0.5061
Epoch 7/10
23/23 [==============================] - 1s 26ms/step - loss: 0.0319 - acc: 0.9891 - val_loss: 1.1544 - val_acc: 0.4980
Epoch 8/10
23/23 [==============================] - 1s 33ms/step - loss: 0.0205 - a

layers 2, drop-out 0.7

In [136]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 4s 59ms/step - loss: 0.7088 - acc: 0.5150 - val_loss: 0.6989 - val_acc: 0.4531
Epoch 2/10
23/23 [==============================] - 1s 32ms/step - loss: 0.6721 - acc: 0.5926 - val_loss: 0.6990 - val_acc: 0.5224
Epoch 3/10
23/23 [==============================] - 1s 43ms/step - loss: 0.5679 - acc: 0.7439 - val_loss: 0.7217 - val_acc: 0.4449
Epoch 4/10
23/23 [==============================] - 1s 26ms/step - loss: 0.3982 - acc: 0.8515 - val_loss: 0.7872 - val_acc: 0.4612
Epoch 5/10
23/23 [==============================] - 1s 26ms/step - loss: 0.2249 - acc: 0.9223 - val_loss: 0.8625 - val_acc: 0.4776
Epoch 6/10
23/23 [==============================] - 1s 26ms/step - loss: 0.1183 - acc: 0.9714 - val_loss: 0.9668 - val_acc: 0.4898
Epoch 7/10
23/23 [==============================] - 1s 29ms/step - loss: 0.0695 - acc: 0.9782 - val_loss: 1.0560 - val_acc: 0.4939
Epoch 8/10
23/23 [==============================] - 1s 33ms/step - loss: 0.0500 - a

layers 3, drop-out 0.3

In [137]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(SimpleRNN(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 6s 91ms/step - loss: 0.7424 - acc: 0.4905 - val_loss: 0.7137 - val_acc: 0.4653
Epoch 2/10
23/23 [==============================] - 1s 42ms/step - loss: 0.6365 - acc: 0.6526 - val_loss: 0.6898 - val_acc: 0.5224
Epoch 3/10
23/23 [==============================] - 1s 56ms/step - loss: 0.4703 - acc: 0.7956 - val_loss: 0.7235 - val_acc: 0.5469
Epoch 4/10
23/23 [==============================] - 1s 52ms/step - loss: 0.2568 - acc: 0.9183 - val_loss: 0.8781 - val_acc: 0.5388
Epoch 5/10
23/23 [==============================] - 1s 47ms/step - loss: 0.1147 - acc: 0.9673 - val_loss: 0.9662 - val_acc: 0.5510
Epoch 6/10
23/23 [==============================] - 1s 44ms/step - loss: 0.0581 - acc: 0.9768 - val_loss: 1.1380 - val_acc: 0.5265
Epoch 7/10
23/23 [==============================] - 1s 47ms/step - loss: 0.0318 - acc: 0.9946 - val_loss: 1.2956 - val_acc: 0.5184
Epoch 8/10
23/23 [==============================] - 1s 40ms/step - loss: 0.0245 - a

layers 3, drop-out 0.7

In [138]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(SimpleRNN(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 15s 416ms/step - loss: 0.7739 - acc: 0.4823 - val_loss: 0.6945 - val_acc: 0.5184
Epoch 2/10
23/23 [==============================] - 3s 146ms/step - loss: 0.7544 - acc: 0.4959 - val_loss: 0.7170 - val_acc: 0.4898
Epoch 3/10
23/23 [==============================] - 3s 113ms/step - loss: 0.7056 - acc: 0.5354 - val_loss: 0.6946 - val_acc: 0.5184
Epoch 4/10
23/23 [==============================] - 2s 94ms/step - loss: 0.6867 - acc: 0.5559 - val_loss: 0.6944 - val_acc: 0.5469
Epoch 5/10
23/23 [==============================] - 2s 84ms/step - loss: 0.6185 - acc: 0.6757 - val_loss: 0.7591 - val_acc: 0.5306
Epoch 6/10
23/23 [==============================] - 2s 85ms/step - loss: 0.4718 - acc: 0.7725 - val_loss: 0.8881 - val_acc: 0.5102
Epoch 7/10
23/23 [==============================] - 2s 69ms/step - loss: 0.3444 - acc: 0.8569 - val_loss: 1.0185 - val_acc: 0.5306
Epoch 8/10
23/23 [==============================] - 1s 63ms/step - loss: 0.2308

# LSTM

layers 2, drop-out 0.3

In [141]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 9s 131ms/step - loss: 0.6939 - acc: 0.5068 - val_loss: 0.6936 - val_acc: 0.4694
Epoch 2/10
23/23 [==============================] - 1s 64ms/step - loss: 0.6924 - acc: 0.5490 - val_loss: 0.6930 - val_acc: 0.4735
Epoch 3/10
23/23 [==============================] - 2s 71ms/step - loss: 0.6894 - acc: 0.5572 - val_loss: 0.6930 - val_acc: 0.4694
Epoch 4/10
23/23 [==============================] - 1s 63ms/step - loss: 0.6682 - acc: 0.6608 - val_loss: 0.6735 - val_acc: 0.5959
Epoch 5/10
23/23 [==============================] - 2s 70ms/step - loss: 0.5746 - acc: 0.7902 - val_loss: 0.7518 - val_acc: 0.5102
Epoch 6/10
23/23 [==============================] - 2s 67ms/step - loss: 0.4164 - acc: 0.8747 - val_loss: 0.6363 - val_acc: 0.6449
Epoch 7/10
23/23 [==============================] - 2s 76ms/step - loss: 0.2561 - acc: 0.9387 - val_loss: 0.6566 - val_acc: 0.6571
Epoch 8/10
23/23 [==============================] - 2s 67ms/step - loss: 0.2141 - 

layers 2, drop-out 0.7

In [142]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 9s 139ms/step - loss: 0.6941 - acc: 0.4823 - val_loss: 0.6931 - val_acc: 0.5224
Epoch 2/10
23/23 [==============================] - 2s 71ms/step - loss: 0.6931 - acc: 0.5163 - val_loss: 0.6927 - val_acc: 0.5306
Epoch 3/10
23/23 [==============================] - 2s 68ms/step - loss: 0.6924 - acc: 0.5300 - val_loss: 0.6926 - val_acc: 0.5469
Epoch 4/10
23/23 [==============================] - 2s 101ms/step - loss: 0.6895 - acc: 0.6022 - val_loss: 0.6911 - val_acc: 0.5429
Epoch 5/10
23/23 [==============================] - 2s 89ms/step - loss: 0.6768 - acc: 0.6185 - val_loss: 0.6838 - val_acc: 0.5592
Epoch 6/10
23/23 [==============================] - 1s 59ms/step - loss: 0.6062 - acc: 0.7602 - val_loss: 0.6566 - val_acc: 0.5755
Epoch 7/10
23/23 [==============================] - 1s 62ms/step - loss: 0.4468 - acc: 0.8515 - val_loss: 0.8101 - val_acc: 0.5184
Epoch 8/10
23/23 [==============================] - 2s 89ms/step - loss: 0.3131 -

layers 3, drop-out 0.3

In [143]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 14s 182ms/step - loss: 0.6942 - acc: 0.4986 - val_loss: 0.6924 - val_acc: 0.5306
Epoch 2/10
23/23 [==============================] - 2s 92ms/step - loss: 0.6935 - acc: 0.4918 - val_loss: 0.6926 - val_acc: 0.5306
Epoch 3/10
23/23 [==============================] - 2s 102ms/step - loss: 0.6934 - acc: 0.4946 - val_loss: 0.6927 - val_acc: 0.5306
Epoch 4/10
23/23 [==============================] - 2s 99ms/step - loss: 0.6932 - acc: 0.5095 - val_loss: 0.6934 - val_acc: 0.4898
Epoch 5/10
23/23 [==============================] - 3s 119ms/step - loss: 0.6935 - acc: 0.4959 - val_loss: 0.6933 - val_acc: 0.5020
Epoch 6/10
23/23 [==============================] - 3s 123ms/step - loss: 0.6926 - acc: 0.5381 - val_loss: 0.6932 - val_acc: 0.4980
Epoch 7/10
23/23 [==============================] - 3s 121ms/step - loss: 0.6916 - acc: 0.5272 - val_loss: 0.6950 - val_acc: 0.4694
Epoch 8/10
23/23 [==============================] - 2s 102ms/step - loss: 0.6

layers 3, drop-out 0.7

In [144]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 14s 195ms/step - loss: 0.6938 - acc: 0.4946 - val_loss: 0.6933 - val_acc: 0.4735
Epoch 2/10
23/23 [==============================] - 2s 105ms/step - loss: 0.6939 - acc: 0.4986 - val_loss: 0.6931 - val_acc: 0.4735
Epoch 3/10
23/23 [==============================] - 3s 135ms/step - loss: 0.6931 - acc: 0.5232 - val_loss: 0.6923 - val_acc: 0.5347
Epoch 4/10
23/23 [==============================] - 3s 136ms/step - loss: 0.6926 - acc: 0.5068 - val_loss: 0.6922 - val_acc: 0.5918
Epoch 5/10
23/23 [==============================] - 4s 195ms/step - loss: 0.6898 - acc: 0.5559 - val_loss: 0.6956 - val_acc: 0.4694
Epoch 6/10
23/23 [==============================] - 3s 134ms/step - loss: 0.6771 - acc: 0.5858 - val_loss: 0.6767 - val_acc: 0.6122
Epoch 7/10
23/23 [==============================] - 3s 119ms/step - loss: 0.5908 - acc: 0.7262 - val_loss: 0.6451 - val_acc: 0.6204
Epoch 8/10
23/23 [==============================] - 3s 142ms/step - loss: 0

# GRU

layers 2, drop-out 0.3

In [146]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 12s 153ms/step - loss: 0.6942 - acc: 0.5068 - val_loss: 0.6927 - val_acc: 0.5429
Epoch 2/10
23/23 [==============================] - 2s 71ms/step - loss: 0.6919 - acc: 0.5490 - val_loss: 0.6916 - val_acc: 0.5429
Epoch 3/10
23/23 [==============================] - 2s 97ms/step - loss: 0.6879 - acc: 0.6322 - val_loss: 0.6892 - val_acc: 0.5837
Epoch 4/10
23/23 [==============================] - 2s 68ms/step - loss: 0.6615 - acc: 0.7289 - val_loss: 0.6734 - val_acc: 0.5918
Epoch 5/10
23/23 [==============================] - 2s 93ms/step - loss: 0.5146 - acc: 0.8474 - val_loss: 0.6493 - val_acc: 0.6327
Epoch 6/10
23/23 [==============================] - 1s 59ms/step - loss: 0.2904 - acc: 0.9114 - val_loss: 0.8925 - val_acc: 0.5347
Epoch 7/10
23/23 [==============================] - 1s 59ms/step - loss: 0.1492 - acc: 0.9537 - val_loss: 0.8082 - val_acc: 0.6286
Epoch 8/10
23/23 [==============================] - 2s 74ms/step - loss: 0.0736 -

layers 2, drop-out 0.7

In [147]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 14s 170ms/step - loss: 0.6938 - acc: 0.4877 - val_loss: 0.6922 - val_acc: 0.5265
Epoch 2/10
23/23 [==============================] - 2s 88ms/step - loss: 0.6889 - acc: 0.5899 - val_loss: 0.6893 - val_acc: 0.6163
Epoch 3/10
23/23 [==============================] - 2s 87ms/step - loss: 0.6714 - acc: 0.6703 - val_loss: 0.6805 - val_acc: 0.5837
Epoch 4/10
23/23 [==============================] - 2s 69ms/step - loss: 0.5640 - acc: 0.8202 - val_loss: 0.6450 - val_acc: 0.6449
Epoch 5/10
23/23 [==============================] - 2s 84ms/step - loss: 0.3349 - acc: 0.8815 - val_loss: 0.6765 - val_acc: 0.6694
Epoch 6/10
23/23 [==============================] - 2s 78ms/step - loss: 0.1600 - acc: 0.9482 - val_loss: 0.7834 - val_acc: 0.6408
Epoch 7/10
23/23 [==============================] - 2s 99ms/step - loss: 0.0838 - acc: 0.9741 - val_loss: 0.8619 - val_acc: 0.6612
Epoch 8/10
23/23 [==============================] - 2s 105ms/step - loss: 0.0514 

layers 3, drop-out 0.3

In [148]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 12s 160ms/step - loss: 0.6944 - acc: 0.4864 - val_loss: 0.6934 - val_acc: 0.4612
Epoch 2/10
23/23 [==============================] - 2s 85ms/step - loss: 0.6926 - acc: 0.5082 - val_loss: 0.6924 - val_acc: 0.5347
Epoch 3/10
23/23 [==============================] - 2s 84ms/step - loss: 0.6863 - acc: 0.5654 - val_loss: 0.6920 - val_acc: 0.5184
Epoch 4/10
23/23 [==============================] - 2s 89ms/step - loss: 0.6218 - acc: 0.7221 - val_loss: 0.6696 - val_acc: 0.5592
Epoch 5/10
23/23 [==============================] - 2s 80ms/step - loss: 0.3756 - acc: 0.8488 - val_loss: 0.6999 - val_acc: 0.6286
Epoch 6/10
23/23 [==============================] - 2s 94ms/step - loss: 0.1814 - acc: 0.9428 - val_loss: 0.7931 - val_acc: 0.6367
Epoch 7/10
23/23 [==============================] - 2s 99ms/step - loss: 0.0912 - acc: 0.9714 - val_loss: 0.9640 - val_acc: 0.6000
Epoch 8/10
23/23 [==============================] - 2s 101ms/step - loss: 0.0494 

layers 3, drop-out 0.7

In [149]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(GRU(32, return_sequences=True))
model.add(Dropout(0.7))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 16s 287ms/step - loss: 0.6941 - acc: 0.4905 - val_loss: 0.6938 - val_acc: 0.4694
Epoch 2/10
23/23 [==============================] - 3s 110ms/step - loss: 0.6912 - acc: 0.5545 - val_loss: 0.6934 - val_acc: 0.4816
Epoch 3/10
23/23 [==============================] - 2s 97ms/step - loss: 0.6860 - acc: 0.5599 - val_loss: 0.6908 - val_acc: 0.5265
Epoch 4/10
23/23 [==============================] - 2s 100ms/step - loss: 0.6379 - acc: 0.6894 - val_loss: 0.6969 - val_acc: 0.5347
Epoch 5/10
23/23 [==============================] - 2s 93ms/step - loss: 0.4074 - acc: 0.8474 - val_loss: 0.7100 - val_acc: 0.5959
Epoch 6/10
23/23 [==============================] - 5s 215ms/step - loss: 0.2179 - acc: 0.9237 - val_loss: 0.8109 - val_acc: 0.6531
Epoch 7/10
23/23 [==============================] - 5s 200ms/step - loss: 0.1093 - acc: 0.9646 - val_loss: 1.0298 - val_acc: 0.5878
Epoch 8/10
23/23 [==============================] - 5s 231ms/step - loss: 0.0

# Bidrectional LSTM

layers 2, drop-out 0.3

In [166]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(10000, 32))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 16s 260ms/step - loss: 0.6944 - acc: 0.4891 - val_loss: 0.6939 - val_acc: 0.4694
Epoch 2/10
23/23 [==============================] - 2s 90ms/step - loss: 0.6939 - acc: 0.4809 - val_loss: 0.6927 - val_acc: 0.5388
Epoch 3/10
23/23 [==============================] - 3s 112ms/step - loss: 0.6926 - acc: 0.5409 - val_loss: 0.6924 - val_acc: 0.5755
Epoch 4/10
23/23 [==============================] - 2s 107ms/step - loss: 0.6909 - acc: 0.5872 - val_loss: 0.6893 - val_acc: 0.5592
Epoch 5/10
23/23 [==============================] - 2s 99ms/step - loss: 0.6739 - acc: 0.6199 - val_loss: 0.6574 - val_acc: 0.6490
Epoch 6/10
23/23 [==============================] - 2s 82ms/step - loss: 0.4388 - acc: 0.8365 - val_loss: 0.6887 - val_acc: 0.6449
Epoch 7/10
23/23 [==============================] - 2s 84ms/step - loss: 0.2200 - acc: 0.9292 - val_loss: 0.9643 - val_acc: 0.6490
Epoch 8/10
23/23 [==============================] - 2s 97ms/step - loss: 0.1187

layers 2, drop-out 0.7

In [165]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(10000, 32))
model.add(Bidirectional(LSTM(32, return_sequences=True))) 
model.add(Dropout(0.7))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 18s 254ms/step - loss: 0.6944 - acc: 0.4619 - val_loss: 0.6929 - val_acc: 0.5510
Epoch 2/10
23/23 [==============================] - 4s 160ms/step - loss: 0.6933 - acc: 0.5123 - val_loss: 0.6933 - val_acc: 0.4694
Epoch 3/10
23/23 [==============================] - 3s 141ms/step - loss: 0.6926 - acc: 0.5109 - val_loss: 0.6916 - val_acc: 0.5551
Epoch 4/10
23/23 [==============================] - 3s 111ms/step - loss: 0.6900 - acc: 0.5518 - val_loss: 0.6895 - val_acc: 0.6204
Epoch 5/10
23/23 [==============================] - 3s 117ms/step - loss: 0.6700 - acc: 0.6390 - val_loss: 0.6700 - val_acc: 0.5796
Epoch 6/10
23/23 [==============================] - 3s 120ms/step - loss: 0.4735 - acc: 0.7984 - val_loss: 0.7266 - val_acc: 0.6449
Epoch 7/10
23/23 [==============================] - 3s 146ms/step - loss: 0.2166 - acc: 0.9360 - val_loss: 0.9019 - val_acc: 0.6653
Epoch 8/10
23/23 [==============================] - 4s 155ms/step - loss: 0

layers 3, drop-out 0.3

In [167]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32, return_sequences=True))) 
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 24s 297ms/step - loss: 0.6946 - acc: 0.4796 - val_loss: 0.6927 - val_acc: 0.5306
Epoch 2/10
23/23 [==============================] - 4s 184ms/step - loss: 0.6935 - acc: 0.4932 - val_loss: 0.6929 - val_acc: 0.5306
Epoch 3/10
23/23 [==============================] - 3s 137ms/step - loss: 0.6928 - acc: 0.5082 - val_loss: 0.6916 - val_acc: 0.5306
Epoch 4/10
23/23 [==============================] - 3s 132ms/step - loss: 0.6927 - acc: 0.5054 - val_loss: 0.6915 - val_acc: 0.5510
Epoch 5/10
23/23 [==============================] - 4s 170ms/step - loss: 0.6857 - acc: 0.5804 - val_loss: 0.6839 - val_acc: 0.5959
Epoch 6/10
23/23 [==============================] - 4s 167ms/step - loss: 0.5540 - acc: 0.7561 - val_loss: 0.6934 - val_acc: 0.6367
Epoch 7/10
23/23 [==============================] - 3s 151ms/step - loss: 0.2587 - acc: 0.9060 - val_loss: 0.9376 - val_acc: 0.6041
Epoch 8/10
23/23 [==============================] - 3s 121ms/step - loss: 0

layers 3, drop-out 0.7

In [168]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.7))
model.add(Bidirectional(LSTM(32, return_sequences=True))) 
model.add(Dropout(0.7))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 27s 547ms/step - loss: 0.6939 - acc: 0.5054 - val_loss: 0.6923 - val_acc: 0.5306
Epoch 2/10
23/23 [==============================] - 3s 140ms/step - loss: 0.6942 - acc: 0.4755 - val_loss: 0.6938 - val_acc: 0.4694
Epoch 3/10
23/23 [==============================] - 4s 161ms/step - loss: 0.6937 - acc: 0.5000 - val_loss: 0.6926 - val_acc: 0.5306
Epoch 4/10
23/23 [==============================] - 4s 173ms/step - loss: 0.6930 - acc: 0.5136 - val_loss: 0.6915 - val_acc: 0.5306
Epoch 5/10
23/23 [==============================] - 3s 137ms/step - loss: 0.6919 - acc: 0.5272 - val_loss: 0.6908 - val_acc: 0.5306
Epoch 6/10
23/23 [==============================] - 4s 155ms/step - loss: 0.6858 - acc: 0.5777 - val_loss: 0.6937 - val_acc: 0.4857
Epoch 7/10
23/23 [==============================] - 4s 159ms/step - loss: 0.6230 - acc: 0.7098 - val_loss: 0.6790 - val_acc: 0.5878
Epoch 8/10
23/23 [==============================] - 3s 150ms/step - loss: 0